In [1]:
import torch
import numpy as np

from torchonometrics.linear import LinearRegression
from torchonometrics.mle import LogisticRegression, PoissonRegression

In [2]:
torch.manual_seed(42)
n, p = 100, 5
X = torch.randn(n, p)
true_coef = torch.randn(p)
y = X @ true_coef + 0.1 * torch.randn(n)

model = LinearRegression()
model.fit(X, y, se="HC1")

model.summary()

LinearRegression Results
coef: tensor([-0.1935,  0.0301, -1.3814, -0.3106,  0.9807])
se: tensor([0.0097, 0.0082, 0.0124, 0.0111, 0.0123])


In [3]:
# Panel data setup
n_firms, n_years = 100, 10
n_obs = n_firms * n_years

# Generate data with firm and year effects
X = torch.randn(n_obs, 3)
firm_ids = torch.repeat_interleave(torch.arange(n_firms), n_years)
year_ids = torch.tile(torch.arange(n_years), (n_firms,))

# Add intercept

# True coefficients and effects
true_coef = torch.tensor([1.5, -0.8, 0.3])
firm_effects = torch.randn(n_firms)[firm_ids]
year_effects = torch.randn(n_years)[year_ids]

y = X @ true_coef + firm_effects + year_effects + 0.1 * torch.randn(n_obs)

# Fit with two-way fixed effects
model = LinearRegression()
model.fit(X, y, fe=[firm_ids, year_ids])
print(f"Coefficients: {model.params['coef']}")

Coefficients: tensor([ 1.4982, -0.7990,  0.2962], dtype=torch.float64)


## logit

In [4]:
# Binary classification data
n, p = 5_000, 4
X = torch.randn(n, p)
X_with_intercept = torch.cat([torch.ones(n, 1), X], dim=1)

# Generate binary outcomes
true_coef = torch.tensor([0.5, 1.0, -0.8, 0.3, 0.2])
logits = X_with_intercept @ true_coef
probs = torch.sigmoid(logits)
y = torch.bernoulli(probs)

# Fit logistic regression
model = LogisticRegression(maxiter=100)
model.fit(X_with_intercept, y)
model.summary()

LogisticRegression Results
Optimizer: LBFGS
Optimization: 12/100 iterations
Final Log-Likelihood: -2721.3655
No. Observations: 5000

Variable     Coef.      Std.Err.   t        P>|t|    [95.0%   Conf. Interval]
----------------------------------------------------------------------
const        0.4891     0.0336     14.553   0.000    0.423    0.555   
x1           0.9586     0.0384     24.985   0.000    0.883    1.034   
x2           -0.7783    0.0371     -20.954  0.000    -0.851   -0.706  
x3           0.2916     0.0334     8.718    0.000    0.226    0.357   
x4           0.1774     0.0329     5.398    0.000    0.113    0.242   


In [5]:
# Predictions
y_pred_proba = model.predict_proba(X_with_intercept)
y_pred = model.predict(X_with_intercept)
torch.unique(y_pred, return_counts=True)

(tensor([0, 1], dtype=torch.int32), tensor([1706, 3294]))

## poisson

In [6]:
torch.manual_seed(42)
n, p = 200, 2
X = torch.randn(n, p)
X_with_intercept = torch.cat([torch.ones(n, 1), X], dim=1)
true_coef = torch.tensor([1.0, 0.5, -0.3])

# Generate Poisson counts
linear_pred = X_with_intercept @ true_coef
lambda_true = torch.exp(linear_pred)
y = torch.poisson(lambda_true)

model = PoissonRegression(maxiter=50)
model.fit(X_with_intercept, y)
model.summary()

PoissonRegression Results
Optimizer: LBFGS
Optimization: 12/50 iterations
Final Log-Likelihood: 95.8605
No. Observations: 200

Variable     Coef.      Std.Err.   t        P>|t|    [95.0%   Conf. Interval]
----------------------------------------------------------------------
const        0.9723     0.0455     21.348   0.000    0.883    1.062   
x1           0.4923     0.0455     10.821   0.000    0.403    0.581   
x2           -0.3467    0.0447     -7.754   0.000    -0.434   -0.259  
